# Gradient Boosting

---

### Installation 

---

### Import Libraries and Dependencies

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
%matplotlib inline

# Turn off warnings
import warnings
warnings.filterwarnings("ignore")

import gc

### Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
def encode(X):
  return X.apply(lambda x: X[x.name] if X[x.name].dtype == 'int64' or X[x.name].dtype == 'float64' else LabelEncoder().fit_transform(X[x.name].astype(str)))

### Helper functions

In [ ]:
# def fill_in(data):
#     for i in data.columns:
#         if data[i].dtype == "object":
#             data[i] = data[i].fillna("")
#         elif (data[i].dtype == "int64" or data[i].dtype == "float64"):
#             data[i] = data[i].fillna(data[i].mean())
#         else:
#             pass
#     return data

In [ ]:
def get_agg(app, agg_list, group_by_column='SK_ID_CURR'):
  app = encode(app)
  app = app.groupby(group_by_column).agg(agg_list)
  app.columns = ['_'.join(col) for col in app.columns.values]
  app.reset_index(inplace=True)
  return app

### Read in CSV as Pandas DataFrame

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
relative_path = '/content/drive/My Drive/Colab Notebooks'

Mounted at /content/drive


In [ ]:
def engineer_features(df):
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    return df

In [ ]:
application_train = pd.read_csv(Path(relative_path + '/home-credit-default-risk/application_train.csv'))
application_train['DAYS_EMPLOYED'] = application_train['DAYS_EMPLOYED'].replace(365243, 0)
application_train['OWN_CAR_AGE'] = application_train['OWN_CAR_AGE'].fillna(0)
# application_train = fill_in(application_train)
application_train = engineer_features(application_train)
application_train = encode(application_train)
application_train.shape

(307511, 127)

In [ ]:
application_test = pd.read_csv(Path(relative_path + '/home-credit-default-risk/application_test.csv'))
application_test['DAYS_EMPLOYED'] = application_test['DAYS_EMPLOYED'].replace(365243, 0)
application_test['OWN_CAR_AGE'] = application_test['OWN_CAR_AGE'].fillna(0)
# application_test = fill_in(application_test)
application_test = engineer_features(application_test)
application_test = encode(application_test)
application_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,...,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE
0,100001,0,0,0,1,0,135000.0,568800.0,20560.5,450000.0,6,6,1,1,1,0.018850,-19241,-2329,-5170.0,-812,0.0,1,1,0,1,0,1,18,2.0,2,2,5,18,0,0,0,0,0,0,28,...,0,0.0392,5,0,0.0,0.0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.121044,0.237342,67500.0,0.152300,0.036147
1,100005,0,1,0,1,0,99000.0,222768.0,17370.0,180000.0,6,6,4,1,1,0.035792,-18064,-4469,-9118.0,-1623,0.0,1,1,0,1,0,0,9,2.0,2,2,0,9,0,0,0,0,0,0,42,...,1,NaN,7,2,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0,0.247398,0.444409,49500.0,0.175455,0.077973
2,100013,0,1,1,1,0,202500.0,663264.0,69777.0,630000.0,7,6,1,1,1,0.019101,-20038,-4458,-2175.0,-3503,5.0,1,1,0,1,0,0,4,2.0,2,2,1,14,0,0,0,0,0,0,54,...,1,NaN,7,2,0.0,0.0,0.0,0.0,-856.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0,0.222477,0.305308,101250.0,0.344578,0.105202
3,100028,0,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,6,6,4,1,1,0.026392,-13976,-1866,-2000.0,-4208,0.0,1,1,0,1,1,0,14,4.0,2,2,6,11,0,0,0,0,0,0,5,...,0,0.3700,4,0,0.0,0.0,0.0,0.0,-1805.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0,0.133515,0.200000,78750.0,0.155614,0.031123
4,100038,0,1,1,0,1,180000.0,625500.0,32067.0,625500.0,6,6,4,1,1,0.010032,-13040,-2191,-4000.0,-4262,16.0,1,1,1,1,0,0,18,3.0,2,2,0,5,0,0,0,0,1,1,5,...,1,NaN,7,2,0.0,0.0,0.0,0.0,-821.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.168021,0.287770,60000.0,0.178150,0.051266


In [ ]:
credit_card_balance = pd.read_csv(Path(relative_path + '/home-credit-default-risk/credit_card_balance.csv'))
credit_card_balance.drop(columns=['SK_ID_PREV'], inplace=True)
# credit_card_balance = fill_in(credit_card_balance)
credit_card_balance.head()

,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [ ]:
credit_card_balance = get_agg(credit_card_balance, ['min', 'max', 'mean', 'sum', 'var'])
credit_card_balance.head()

,SK_ID_CURR,MONTHS_BALANCE_min,MONTHS_BALANCE_max,MONTHS_BALANCE_mean,MONTHS_BALANCE_sum,MONTHS_BALANCE_var,AMT_BALANCE_min,AMT_BALANCE_max,AMT_BALANCE_mean,AMT_BALANCE_sum,AMT_BALANCE_var,AMT_CREDIT_LIMIT_ACTUAL_min,AMT_CREDIT_LIMIT_ACTUAL_max,AMT_CREDIT_LIMIT_ACTUAL_mean,AMT_CREDIT_LIMIT_ACTUAL_sum,AMT_CREDIT_LIMIT_ACTUAL_var,AMT_DRAWINGS_ATM_CURRENT_min,AMT_DRAWINGS_ATM_CURRENT_max,AMT_DRAWINGS_ATM_CURRENT_mean,AMT_DRAWINGS_ATM_CURRENT_sum,AMT_DRAWINGS_ATM_CURRENT_var,AMT_DRAWINGS_CURRENT_min,AMT_DRAWINGS_CURRENT_max,AMT_DRAWINGS_CURRENT_mean,AMT_DRAWINGS_CURRENT_sum,AMT_DRAWINGS_CURRENT_var,AMT_DRAWINGS_OTHER_CURRENT_min,AMT_DRAWINGS_OTHER_CURRENT_max,AMT_DRAWINGS_OTHER_CURRENT_mean,AMT_DRAWINGS_OTHER_CURRENT_sum,AMT_DRAWINGS_OTHER_CURRENT_var,AMT_DRAWINGS_POS_CURRENT_min,AMT_DRAWINGS_POS_CURRENT_max,AMT_DRAWINGS_POS_CURRENT_mean,AMT_DRAWINGS_POS_CURRENT_sum,AMT_DRAWINGS_POS_CURRENT_var,AMT_INST_MIN_REGULARITY_min,AMT_INST_MIN_REGULARITY_max,AMT_INST_MIN_REGULARITY_mean,AMT_INST_MIN_REGULARITY_sum,...,CNT_DRAWINGS_ATM_CURRENT_min,CNT_DRAWINGS_ATM_CURRENT_max,CNT_DRAWINGS_ATM_CURRENT_mean,CNT_DRAWINGS_ATM_CURRENT_sum,CNT_DRAWINGS_ATM_CURRENT_var,CNT_DRAWINGS_CURRENT_min,CNT_DRAWINGS_CURRENT_max,CNT_DRAWINGS_CURRENT_mean,CNT_DRAWINGS_CURRENT_sum,CNT_DRAWINGS_CURRENT_var,CNT_DRAWINGS_OTHER_CURRENT_min,CNT_DRAWINGS_OTHER_CURRENT_max,CNT_DRAWINGS_OTHER_CURRENT_mean,CNT_DRAWINGS_OTHER_CURRENT_sum,CNT_DRAWINGS_OTHER_CURRENT_var,CNT_DRAWINGS_POS_CURRENT_min,CNT_DRAWINGS_POS_CURRENT_max,CNT_DRAWINGS_POS_CURRENT_mean,CNT_DRAWINGS_POS_CURRENT_sum,CNT_DRAWINGS_POS_CURRENT_var,CNT_INSTALMENT_MATURE_CUM_min,CNT_INSTALMENT_MATURE_CUM_max,CNT_INSTALMENT_MATURE_CUM_mean,CNT_INSTALMENT_MATURE_CUM_sum,CNT_INSTALMENT_MATURE_CUM_var,NAME_CONTRACT_STATUS_min,NAME_CONTRACT_STATUS_max,NAME_CONTRACT_STATUS_mean,NAME_CONTRACT_STATUS_sum,NAME_CONTRACT_STATUS_var,SK_DPD_min,SK_DPD_max,SK_DPD_mean,SK_DPD_sum,SK_DPD_var,SK_DPD_DEF_min,SK_DPD_DEF_max,SK_DPD_DEF_mean,SK_DPD_DEF_sum,SK_DPD_DEF_var
0,100006,-6,-1,-3.5,-21,3.5,0.0,0.00,0.000000,0.000,0.000000e+00,270000,270000,270000.000000,1620000,0.000000e+00,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.000000,0.0,0.000000e+00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.000000,0.000,...,NaN,NaN,NaN,0.0,NaN,0,0,0.000000,0,0.000000,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.000000,0.0,0.000000,0,0,0.000000,0,0.000000,0,0,0.000000,0,0.000000,0,0,0.000000,0,0.000000
1,100011,-75,-2,-38.5,-2849,462.5,0.0,189000.00,54482.111149,4031676.225,4.641321e+09,90000,180000,164189.189189,12150000,1.189060e+09,0.0,180000.0,2432.432432,180000.0,4.378378e+08,0.0,180000.0,2432.432432,180000.0,4.378378e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9000.0,3956.221849,288804.195,...,0.0,4.0,0.054054,4.0,0.216216,0,4,0.054054,4,0.216216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,33.0,25.767123,1881.0,105.847793,0,0,0.000000,0,0.000000,0,0,0.000000,0,0.000000,0,0,0.000000,0,0.000000
2,100013,-96,-1,-48.5,-4656,776.0,0.0,161420.22,18159.919219,1743352.245,1.869473e+09,45000,157500,131718.750000,12645000,2.259252e+09,0.0,157500.0,6350.000000,571500.0,8.249688e+08,0.0,157500.0,5953.125000,571500.0,7.752530e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7875.0,1454.539551,129454.020,...,0.0,7.0,0.255556,23.0,1.405868,0,7,0.239583,23,1.320943,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,22.0,18.719101,1666.0,34.249745,0,0,0.000000,0,0.000000,0,1,0.010417,1,0.010417,0,1,0.010417,1,0.010417
3,100021,-18,-2,-10.0,-170,25.5,0.0,0.00,0.000000,0.000,0.000000e+00,675000,675000,675000.000000,11475000,0.000000e+00,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.000000,0.0,0.000000e+00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.000000,0.000,...,NaN,NaN,NaN,0.0,NaN,0,0,0.000000,0,0.000000,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.000000,0.0,0.000000,0,2,1.176471,20,1.029412,0,0,0.000000,0,0.000000,0,0,0.000000,0,0.000000
4,100023,-11,-4,-7.5,-60,6.0,0.0,0.00,0.000000,0.000,0.000000e+00,45000,225000,135000.000000,1080000,9.257143e+09,NaN,NaN,NaN,0

In [ ]:
merged = pd.merge(left=application_train, right=credit_card_balance, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')
merged_test = pd.merge(left=application_test, right=credit_card_balance, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')

del application_train
del credit_card_balance
gc.collect()

0

In [ ]:
prev_app = pd.read_csv(Path(relative_path + '/home-credit-default-risk/previous_application.csv'))
prev_app.replace(365243, np.nan, inplace= True)
prev_app.drop(columns=['SK_ID_PREV'], inplace=True)
# prev_app = fill_in(prev_app)
prev_app['PREV_CREDIT_APPL_RATIO'] = prev_app['AMT_CREDIT']/prev_app['AMT_APPLICATION']
prev_app['PREV_ANNUITY_APPL_RATIO'] = prev_app['AMT_ANNUITY']/prev_app['AMT_APPLICATION']
prev_app['PREV_GOODS_APPL_RATIO'] = prev_app['AMT_GOODS_PRICE']/prev_app['AMT_APPLICATION']
prev_app.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,PREV_CREDIT_APPL_RATIO,PREV_ANNUITY_APPL_RATIO,PREV_GOODS_APPL_RATIO
0,271877.0,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,NaN,-42.0,300.0,-42.0,-37.0,0.0,1.00000,0.100929,1.0
1,108129.0,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,NaN,-134.0,916.0,NaN,NaN,1.0,1.11880,0.041463,1.0
2,122040.0,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,NaN,-271.0,59.0,NaN,NaN,1.0,1.21284,0.133873,1.0
3,176158.0,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,NaN,-482.0,-152.0,-182.0,-177.0,1.0,1.04620,0.104536,1.0
4,202054.0,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN,1.19720,0.094591,1.0


In [ ]:
prev_app_approved = get_agg(prev_app[prev_app['NAME_CONTRACT_STATUS'] == 'Approved'], ['min', 'max', 'mean'])
merged = pd.merge(left=merged, right=prev_app_approved, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')
merged_test = pd.merge(left=merged_test, right=prev_app_approved, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')

del prev_app_approved
gc.collect()

0

In [ ]:
prev_app_refused = get_agg(prev_app[prev_app['NAME_CONTRACT_STATUS'] != 'Approved'], ['min', 'max', 'mean'])
merged = pd.merge(left=merged, right=prev_app_refused, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')
merged_test = pd.merge(left=merged_test, right=prev_app_refused, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')

del prev_app_refused
del prev_app
gc.collect()

0

In [ ]:
bureau = pd.read_csv(relative_path + '/home-credit-default-risk/bureau.csv')
# bureau = fill_in(bureau)
bureau_balance = pd.read_csv(relative_path + '/home-credit-default-risk/bureau_balance.csv')
# bureau_balance = fill_in(bureau_balance)
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [ ]:
bureau_balance = get_agg(bureau_balance, ['min', 'max', 'size'], group_by_column='SK_ID_BUREAU')
bureau_all = pd.merge(left=bureau, right=bureau_balance, how='left', left_on='SK_ID_BUREAU', right_on='SK_ID_BUREAU')

del bureau_balance
gc.collect()

bureau.drop(columns=['SK_ID_BUREAU'], inplace=True)
bureau_all.drop(columns=['SK_ID_BUREAU'], inplace=True)

In [ ]:
bureau_all_active = get_agg(bureau[bureau['CREDIT_ACTIVE'] == 'Active'], ['min', 'max', 'mean'])
merged = pd.merge(left=merged, right=bureau_all_active, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')
merged_test = pd.merge(left=merged_test, right=bureau_all_active, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')

del bureau_all_active
gc.collect()

0

In [ ]:
bureau_all_non_active = get_agg(bureau[bureau['CREDIT_ACTIVE'] != 'Active'], ['min', 'max', 'mean'])
merged = pd.merge(left=merged, right=bureau_all_non_active, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')
merged_test = pd.merge(left=merged_test, right=bureau_all_non_active, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')

del bureau_all_non_active
gc.collect()

0

In [ ]:
installments_payment = pd.read_csv(relative_path + '/home-credit-default-risk/installments_payments.csv')
installments_payment.drop(columns=['SK_ID_PREV'], inplace=True)
# installments_payment = fill_in(installments_payment)
installments_payment['PAYMENT_PERC'] = installments_payment['AMT_PAYMENT'] / installments_payment['AMT_INSTALMENT']
installments_payment['PAYMENT_DIFF'] = installments_payment['AMT_INSTALMENT'] - installments_payment['AMT_PAYMENT']
installments_payment['DAYS_PAST_DUE'] = installments_payment['DAYS_ENTRY_PAYMENT'] - installments_payment['DAYS_INSTALMENT']
installments_payment['DAYS_PAST_DUE'].apply(lambda x: max(0, x))
installments_payment['DAYS_BEFORE_DUE'] = installments_payment['DAYS_INSTALMENT'] - installments_payment['DAYS_ENTRY_PAYMENT']
installments_payment['DAYS_BEFORE_DUE'].apply(lambda x: max(0, x))
installments_payment.head()

,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,PAYMENT_PERC,PAYMENT_DIFF,DAYS_PAST_DUE,DAYS_BEFORE_DUE
0,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360,1.000000,0.000,-7.0,7.0
1,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525,1.000000,0.000,0.0,0.0
2,193053,2.0,1,-63.0,-63.0,25425.000,25425.000,1.000000,0.000,0.0,0.0
3,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130,1.000000,0.000,-8.0,8.0
4,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585,0.997942,4.455,17.0,-17.0


In [ ]:
installments_payment = get_agg(installments_payment, ['max', 'mean', 'sum', 'var'])

In [ ]:
merged = pd.merge(left=merged, right=installments_payment, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')
merged_test = pd.merge(left=merged_test, right=installments_payment, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')

del installments_payment
gc.collect()

125

In [ ]:
pos_cash_balance = pd.read_csv(relative_path + '/home-credit-default-risk/POS_CASH_balance.csv')
pos_cash_balance.drop(columns=['SK_ID_PREV'], inplace=True)
# pos_cash_balance = fill_in(pos_cash_balance)
pos_cash_balance.head()

,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,182943,-31,48.0,45.0,Active,0,0
1,367990,-33,36.0,35.0,Active,0,0
2,397406,-32,12.0,9.0,Active,0,0
3,269225,-35,48.0,42.0,Active,0,0
4,334279,-35,36.0,35.0,Active,0,0


In [ ]:
pos_cash_balance = get_agg(pos_cash_balance, ['max', 'mean'])

In [ ]:
merged = pd.merge(left=merged, right=pos_cash_balance, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')
merged_test = pd.merge(left=merged_test, right=pos_cash_balance, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')

del pos_cash_balance
gc.collect()

50

In [ ]:
merged.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,DAYS_ENTRY_PAYMENT_max,DAYS_ENTRY_PAYMENT_mean,DAYS_ENTRY_PAYMENT_sum,DAYS_ENTRY_PAYMENT_var,AMT_INSTALMENT_max,AMT_INSTALMENT_mean,AMT_INSTALMENT_sum,AMT_INSTALMENT_var,AMT_PAYMENT_max,AMT_PAYMENT_mean,AMT_PAYMENT_sum,AMT_PAYMENT_var,PAYMENT_PERC_max,PAYMENT_PERC_mean,PAYMENT_PERC_sum,PAYMENT_PERC_var,PAYMENT_DIFF_max,PAYMENT_DIFF_mean,PAYMENT_DIFF_sum,PAYMENT_DIFF_var,DAYS_PAST_DUE_max,DAYS_PAST_DUE_mean,DAYS_PAST_DUE_sum,DAYS_PAST_DUE_var,DAYS_BEFORE_DUE_max,DAYS_BEFORE_DUE_mean,DAYS_BEFORE_DUE_sum,DAYS_BEFORE_DUE_var,MONTHS_BALANCE_max_y,MONTHS_BALANCE_mean_y,CNT_INSTALMENT_max,CNT_INSTALMENT_mean,CNT_INSTALMENT_FUTURE_max,CNT_INSTALMENT_FUTURE_mean,NAME_CONTRACT_STATUS_max_y,NAME_CONTRACT_STATUS_mean_y,SK_DPD_max_y,SK_DPD_mean_y,SK_DPD_DEF_max_y,SK_DPD_DEF_mean_y
0,100002,1,0,1,0,1,0,202500.0,406597.5,24700.5,351000.0,6,7,4,3,1,0.018801,-9461,-637,-3648.0,-2120,0.0,1,1,0,1,1,0,8,1.0,2,2,6,10,0,0,0,0,0,0,...,-49.0,-315.421053,-5993.0,29604.257310,53093.745,11559.247105,219625.695,1.011641e+08,53093.745,11559.247105,219625.695,1.011641e+08,1.0,1.000000,19.0,0.000000,0.000,0.000000,0.000,0.000000e+00,-12.0,-20.421053,-388.0,24.257310,31.0,20.421053,388.0,24.257310,-1.0,-10.000000,24.0,24.000000,24.0,15.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1,100003,0,0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,1,4,1,1,1,0.003541,-16765,-1188,-1186.0,-291,0.0,1,1,0,1,1,0,3,2.0,1,1,1,11,0,0,0,0,0,0,...,-544.0,-1385.320000,-34633.0,573541.810000,560835.360,64754.586000,1618864.650,1.221966e+10,560835.360,64754.586000,1618864.650,1.221966e+10,1.0,1.000000,25.0,0.000000,0.000,0.000000,0.000,0.000000e+00,-1.0,-7.160000,-179.0,13.890000,14.0,7.160000,179.0,13.890000,-18.0,-43.785714,12.0,10.107143,12.0,5.785714,4.0,0.285714,0.0,0.0,0.0,0.0
2,100004,0,1,1,1,1,0,67500.0,135000.0,6750.0,135000.0,6,7,4,3,1,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,8,1.0,2,2,1,9,0,0,0,0,0,0,...,-727.0,-761.666667,-2285.0,1157.333333,10573.965,7096.155000,21288.465,9.071372e+06,10573.965,7096.155000,21288.465,9.071372e+06,1.0,1.000000,3.0,0.000000,0.000,0.000000,0.000,0.000000e+00,-3.0,-7.666667,-23.0,17.333333,11.0,7.666667,23.0,17.333333,-24.0,-25.500000,4.0,3.750000,4.0,2.250000,4.0,1.000000,0.0,0.0,0.0,0.0
3,100006,0,0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,6,7,4,0,1,0.008019,-19005,-3039,-9833.0,-2437,0.0,1,1,0,1,0,0,8,2.0,2,2,6,17,0,0,0,0,0,0,...,-12.0,-271.625000,-4346.0,39222.116667,691786.890,62947.088438,1007153.415,2.825681e+10,691786.890,62947.088438,1007153.415,2.825681e+10,1.0,1.000000,16.0,0.000000,0.000,0.000000,0.000,0.000000e+00,-1.0,-19.375000,-310.0,645.050000,77.0,19.375000,310.0,645.050000,-1.0,-9.619048,48.0,12.000000,48.0,8.650000,6.0,0.666667,0.0,0.0,0.0,0.0
4,100007,0,0,1,0,1,0,121500.0,513000.0,21865.5,513000.0,6,7,4,3,1,0.028663,-19932,-3038,-4311.0,-3458,0.0,1,1,0,1,0,0,3,1.0,2,2,4,11,0,0,0,0,1,1,...,-14.0,-1032.242424,-68128.0,461933.971096,22678.785,12666.444545,835985.340,6.166821e+07,22678.785,12214.060227,806127.975,6.477128e+07,1.0,0.954545,63.0,0.043995,22655.655,452.384318,29857.365,8.084830e+06,12.0,-3.636364,-240.0,63.865734,31.0,3.636364,240.0,63.865734,-1.0,-33.636364,24.0,15.333333,24.0,8.969697,7.0,0.287879,0.0,0.0,0.0,0.0


In [ ]:
y = merged["TARGET"]

In [ ]:
X = merged.drop(columns=['SK_ID_CURR', 'TARGET'])
X.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,...,DAYS_ENTRY_PAYMENT_max,DAYS_ENTRY_PAYMENT_mean,DAYS_ENTRY_PAYMENT_sum,DAYS_ENTRY_PAYMENT_var,AMT_INSTALMENT_max,AMT_INSTALMENT_mean,AMT_INSTALMENT_sum,AMT_INSTALMENT_var,AMT_PAYMENT_max,AMT_PAYMENT_mean,AMT_PAYMENT_sum,AMT_PAYMENT_var,PAYMENT_PERC_max,PAYMENT_PERC_mean,PAYMENT_PERC_sum,PAYMENT_PERC_var,PAYMENT_DIFF_max,PAYMENT_DIFF_mean,PAYMENT_DIFF_sum,PAYMENT_DIFF_var,DAYS_PAST_DUE_max,DAYS_PAST_DUE_mean,DAYS_PAST_DUE_sum,DAYS_PAST_DUE_var,DAYS_BEFORE_DUE_max,DAYS_BEFORE_DUE_mean,DAYS_BEFORE_DUE_sum,DAYS_BEFORE_DUE_var,MONTHS_BALANCE_max_y,MONTHS_BALANCE_mean_y,CNT_INSTALMENT_max,CNT_INSTALMENT_mean,CNT_INSTALMENT_FUTURE_max,CNT_INSTALMENT_FUTURE_mean,NAME_CONTRACT_STATUS_max_y,NAME_CONTRACT_STATUS_mean_y,SK_DPD_max_y,SK_DPD_mean_y,SK_DPD_DEF_max_y,SK_DPD_DEF_mean_y
0,0,1,0,1,0,202500.0,406597.5,24700.5,351000.0,6,7,4,3,1,0.018801,-9461,-637,-3648.0,-2120,0.0,1,1,0,1,1,0,8,1.0,2,2,6,10,0,0,0,0,0,0,5,0.083037,...,-49.0,-315.421053,-5993.0,29604.257310,53093.745,11559.247105,219625.695,1.011641e+08,53093.745,11559.247105,219625.695,1.011641e+08,1.0,1.000000,19.0,0.000000,0.000,0.000000,0.000,0.000000e+00,-12.0,-20.421053,-388.0,24.257310,31.0,20.421053,388.0,24.257310,-1.0,-10.000000,24.0,24.000000,24.0,15.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1,0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,1,4,1,1,1,0.003541,-16765,-1188,-1186.0,-291,0.0,1,1,0,1,1,0,3,2.0,1,1,1,11,0,0,0,0,0,0,39,0.311267,...,-544.0,-1385.320000,-34633.0,573541.810000,560835.360,64754.586000,1618864.650,1.221966e+10,560835.360,64754.586000,1618864.650,1.221966e+10,1.0,1.000000,25.0,0.000000,0.000,0.000000,0.000,0.000000e+00,-1.0,-7.160000,-179.0,13.890000,14.0,7.160000,179.0,13.890000,-18.0,-43.785714,12.0,10.107143,12.0,5.785714,4.0,0.285714,0.0,0.0,0.0,0.0
2,1,1,1,1,0,67500.0,135000.0,6750.0,135000.0,6,7,4,3,1,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,8,1.0,2,2,1,9,0,0,0,0,0,0,11,NaN,...,-727.0,-761.666667,-2285.0,1157.333333,10573.965,7096.155000,21288.465,9.071372e+06,10573.965,7096.155000,21288.465,9.071372e+06,1.0,1.000000,3.0,0.000000,0.000,0.000000,0.000,0.000000e+00,-3.0,-7.666667,-23.0,17.333333,11.0,7.666667,23.0,17.333333,-24.0,-25.500000,4.0,3.750000,4.0,2.250000,4.0,1.000000,0.0,0.0,0.0,0.0
3,0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,6,7,4,0,1,0.008019,-19005,-3039,-9833.0,-2437,0.0,1,1,0,1,0,0,8,2.0,2,2,6,17,0,0,0,0,0,0,5,NaN,...,-12.0,-271.625000,-4346.0,39222.116667,691786.890,62947.088438,1007153.415,2.825681e+10,691786.890,62947.088438,1007153.415,2.825681e+10,1.0,1.000000,16.0,0.000000,0.000,0.000000,0.000,0.000000e+00,-1.0,-19.375000,-310.0,645.050000,77.0,19.375000,310.0,645.050000,-1.0,-9.619048,48.0,12.000000,48.0,8.650000,6.0,0.666667,0.0,0.0,0.0,0.0
4,0,1,0,1,0,121500.0,513000.0,21865.5,513000.0,6,7,4,3,1,0.028663,-19932,-3038,-4311.0,-3458,0.0,1,1,0,1,0,0,3,1.0,2,2,4,11,0,0,0,0,1,1,37,NaN,...,-14.0,-1032.242424,-68128.0,461933.971096,22678.785,12666.444545,835985.340,6.166821e+07,22678.785,12214.060227,806127.975,6.477128e+07,1.0,0.954545,63.0,0.043995,22655.655,452.384318,29857.365,8.084830e+06,12.0,-3.636364,-240.0,63.865734,31.0,3.636364,240.0,63.865734,-1.0,-33.636364,24.0,15.333333,24.0,8.969697,7.0,0.287879,0.0,0.0,0.0,0.0


In [ ]:
# del merged
# gc.collect()

### Scaling

In [ ]:
# from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Use the MinMaxScaler to scale data between 0 and 1.
# scaler = MinMaxScaler()
# X_scaled = scaler.fit_transform(X)

In [ ]:
# X_scaled[0]

### Split into train and test

In [ ]:
# from sklearn.feature_selection import RFE

# from lightgbm import LGBMClassifier
# rfe_estimator = LGBMClassifier(n_jobs=4,
#                                objective='binary',
#                                n_estimators=50,
#                                learning_rate=0.2,
#                                verbose=500)

# X = X.apply(lambda x: x.fillna(0))
# X = X.apply(lambda x: x.replace([np.nan, np.inf, -np.inf], 0))
# rfe = RFE(estimator=rfe_estimator)
# X, y = rfe.fit_transform(X, y)

In [ ]:
# X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=78)

In [ ]:
X_train.shape

(230633, 600)

In [ ]:
y_train

187977    0
74944     0
14913     0
103222    0
306080    0
         ..
168477    0
256696    1
104758    0
235415    0
16854     0
Name: TARGET, Length: 230633, dtype: int64

### Oversampling

In [ ]:
# from collections import Counter
# Counter(y_train)

In [ ]:
# from imblearn.over_sampling import SMOTE

# X_train, y_train = SMOTE(random_state=78, sampling_strategy=0.25).fit_resample(X_train, y_train)

---

### Train Model

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
# Creat a Random Hyper parameter grid to assist in selecting the best parameteres for this model
# from sklearn.model_selection import RandomizedSearchCV

# n_estimators = [int(x) for x in np.linspace(start = 20, stop = 100, num = 8)]
# learning_rate = [x * 0.02 for x in range(1, 11)]
# num_leaves = [int(x) for x in np.linspace(100, 2500, num = 8)]
# max_bin = [100, 150, 200]
# num_iteration = [int(x) for x in np.linspace(100, 900, num = 8)]

# random_grid = {'n_estimators': n_estimators,
#                'learning_rate': learning_rate,
#                'num_leaves': num_leaves,
#                'max_bin': max_bin,
#                'num_iteration': num_iteration}

In [ ]:
# rs_cv = RandomizedSearchCV(estimator=LGBMClassifier(), param_distributions=random_grid, cv = 5, n_iter=100,verbose=1)
# rs_cv.fit(X_train, y_train, verbose=-1)

In [ ]:
# rs_cv.best_params_

In [ ]:
# !pip3 install pandas==1.1.5

In [ ]:
prediction_df = pd.DataFrame({})
prediction_proba = pd.DataFrame({})

from sklearn.model_selection import StratifiedKFold
# from sklearn.pipeline import Pipeline
# from imblearn.pipeline import Pipeline
# from imblearn.over_sampling import SMOTE

def get_LGBM_classifier():
  return LGBMClassifier(n_jobs=4,
                        objective='binary',
                        n_estimators=12000,
                        learning_rate=0.02,
                        num_leaves=31,
                        subsample=0.8,
                        max_depth=8,
                        silent=-1,
                        verbose=-1)

folds = StratifiedKFold(n_splits= 5, shuffle=True, random_state=78)

for n_fold, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
  train_x, train_y = X.iloc[train_idx], y.iloc[train_idx]
  valid_x, valid_y = X.iloc[valid_idx], y.iloc[valid_idx]

  # print(train_x.columns[train_x.isna().any()].tolist())
  # print(train_y.isna().any())

  # smote = SMOTE(random_state=78)
  # train_x, train_y = smote.fit_resample(train_x, train_y)

  # model = get_LGBM_classifier()
  # pipeline = Pipeline(steps=[
  #                           #  ('oversampling',smote), 
  #                           #  ('feature_selection',rfe), 
  #                            ('training',model)
  #                            ])
  # pipeline.fit(train_x, 
  #              train_y, 
  #              training__eval_set=[(train_x, train_y), (valid_x, valid_y)], 
  #              training__eval_metric= 'auc', 
  #              training__verbose= 200, 
  #              training__early_stopping_rounds= 100)

  classifier = get_LGBM_classifier()
  classifier.fit(train_x, 
                 train_y,
                 eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                 eval_metric= 'auc', 
                 verbose= 200, 
                 early_stopping_rounds= 100)
  
  prediction_df['Prediction_' + str(n_fold)] = classifier.predict(X_test)
  # kaggle submission
  prediction_proba['Prediction_' + str(n_fold)] = classifier.predict_proba(merged_test.drop(['SK_ID_CURR'], axis=1))[:, 1]

Training until validation scores don't improve for 100 rounds.
[200]	training's auc: 0.798897	training's binary_logloss: 0.235037	valid_1's auc: 0.768698	valid_1's binary_logloss: 0.243822
[400]	training's auc: 0.826067	training's binary_logloss: 0.22418	valid_1's auc: 0.779138	valid_1's binary_logloss: 0.239833
[600]	training's auc: 0.8458	training's binary_logloss: 0.216519	valid_1's auc: 0.782083	valid_1's binary_logloss: 0.238738
[800]	training's auc: 0.862363	training's binary_logloss: 0.210033	valid_1's auc: 0.782992	valid_1's binary_logloss: 0.238434
[1000]	training's auc: 0.876246	training's binary_logloss: 0.204233	valid_1's auc: 0.78368	valid_1's binary_logloss: 0.23826
[1200]	training's auc: 0.888532	training's binary_logloss: 0.198842	valid_1's auc: 0.783879	valid_1's binary_logloss: 0.238133
Early stopping, best iteration is:
[1261]	training's auc: 0.891815	training's binary_logloss: 0.197281	valid_1's auc: 0.783985	valid_1's binary_logloss: 0.238086
Training until validat

In [ ]:
# classifier = LGBMClassifier(n_jobs=4,
#                             objective='binary',
#                             n_estimators=12000,
#                             learning_rate=0.02,
#                             num_leaves=31,
#                             subsample=0.8,
#                             max_depth=8,
#                             silent=-1,
#                             verbose=-1)
# classifier.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], 
#         eval_metric= 'auc', verbose= 200, early_stopping_rounds= 100)

In [ ]:
# from lightgbm import plot_importance

# feature_importance = plot_importance(classifier, figsize=(24, 32))
# feature_importance

In [ ]:
# feature_importance.get_figure().savefig(relative_path + "/feature_importance.png", transparent=False, facecolor='w', edgecolor='w')

---

### Make predictions

In [ ]:
# Changing the threshold
# def make_predictions_based_on_proba(p):
#   return 1 if p > 0.5 else 0

# proba = classifier.predict_proba(X_test)[:, 1]

# prediction_df = pd.DataFrame({"Proba": proba, "Actual": y_test.ravel()})
# prediction_df['Prediction'] = prediction_df['Proba'].apply(lambda p: 1 if p > 0.35 else 0)
# predictions = prediction_df['Prediction'].values
# predictions

In [ ]:
# predictions = classifier.predict(X_test)
# predictions

In [ ]:
predictions = prediction_df.any(axis=1)
predictions = predictions.apply(lambda x: 1 if x else 0)
predictions

0        0
1        1
2        0
3        0
4        1
        ..
76873    0
76874    0
76875    0
76876    0
76877    0
Length: 76878, dtype: int64

---

### Evaluate Model

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
# Generate the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying results
display(cm_df)
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

# Generate classification report
print("Classification Report")
print(classification_report(y_test, predictions))

,Predicted 0,Predicted 1
Actual 0,70454,295
Actual 1,5324,805


Accuracy Score : 0.9269101693592445
Classification Report
              precision    recall  f1-score   support

           0       0.93      1.00      0.96     70749
           1       0.73      0.13      0.22      6129

    accuracy                           0.93     76878
   macro avg       0.83      0.56      0.59     76878
weighted avg       0.91      0.93      0.90     76878



# Kaggle Submission

In [ ]:
# merged_test.head()

In [ ]:
# preds = classifier.predict_proba(merged_test.drop(['SK_ID_CURR'], axis=1))[:, 1]
# preds.shape

In [ ]:
preds = prediction_proba.max(axis=1)

In [ ]:
application_test['TARGET'] = preds
application_test[['SK_ID_CURR', 'TARGET']].to_csv(relative_path + '/result.csv', index=False)